<a target="_blank" href="https://colab.research.google.com/github/cohere-ai/notebooks/blob/main/notebooks/llmu/Building_a_Chatbot.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Building a Chatbot

In this notebook, you’ll build a chatbot using Cohere’s Chat endpoint. By the end of this notebook, you’ll be able to build a simple chatbot that can respond to user messages and maintain the context of the conversation.

*Read the accompanying [blog post here](https://docs.cohere.com/docs/building-a-chatbot).*

## Overview

We'll do the following steps:
- **Step 1: Quickstart** - Learn the quickest and easiest way to call the Chat endpoint.
- **Step 2: Adding a preamble** - Steer a chatbot's response toward certain styles, personas, or other characteristics.
- **Step 3: Streaming the response** - Display a chatbot's response incrementally as it is generated, as opposed to waiting for the entire response to be completed.
- **Step 4: Building the Chat History** - Explore two different options for getting the chatbot to maintain the context of the conversation.

## Setup

We'll start by installing the tools we'll need and then importing them.

In [1]:
! pip install cohere -q

In [ ]:
import cohere
co = cohere.ClientV2("COHERE_API_KEY") # Get your free API key: https://dashboard.cohere.com/api-keys

## Step 1: Quickstart

To get started with Chat, we add a user message to the `messages` parameter and define the model type in the `model` parameter. We then call the Chat endpoint through the client we created earlier.

The response contains several objects. For simplicity, what we want right now is the `text` object.

Here's an example of the assistant responding to a new hire's query asking for help to make introductions.

In [27]:
# Add the user message
message = "I'm joining a new startup called Co1t today. Could you help me write an introduction message for my teammates."

# Generate the response
response = co.chat(model="command-r-plus-08-2024",
                   messages=[{'role':'user', 'content': message}])

print(response.message.content[0].text)

Here's a draft of an introduction message for your new colleagues at Co1t:

"Hello everyone!

My name is [Your Name], and I am thrilled to introduce myself as the newest member of the Co1t team! Starting a new journey at a startup has been an exciting prospect, and I am eager to contribute to the company's success.

A little about myself: I have a background in [Your Educational or Professional Background], and my expertise lies in [Your Skills or Areas of Expertise]. I am passionate about [Mention any relevant interests or hobbies related to the startup's industry]. I believe that my experience and enthusiasm will allow me to quickly integrate into the dynamic culture here.

I am looking forward to meeting and collaborating with all of you. As a team, I am confident that we can achieve great milestones and create innovative solutions together. Please feel free to reach out if you need any assistance or if you'd like to grab a coffee and chat! I'm all for building strong connections an

## Step 2: Creating a custom system message

Optionally, you can add a system message to help steer a chatbot’s response toward certain characteristics.

For example, if we want the chatbot to adopt a formal style, the preamble can be used to encourage the generation of more business-like and professional responses.

The recommended approach is to use two H2 Markdown headers: "Task and Context" and "Style Guide" in the exact order.

In the example below, the preamble provides context for the assistant's task (task and context) and encourages the generation of rhymes as much as possible (style guide).

In [28]:
# Add the user message
message = "I'm joining a new startup called Co1t today. Could you help me write an introduction message for my teammates."

# Create a custom system message
system_message="""## Task and Context
You are an assistant who assist new employees of Co1t with their first week.

## Style Guide
Try to speak in rhymes as much as possible. Be professional."""

# Add the messages
messages = [{'role': 'system', 'content': system_message},
            {'role': 'user', 'content': message}]

# Generate the response
response = co.chat(model="command-r-plus-08-2024",
                   messages=messages)

print(response.message.content[0].text)

Here's a little note, to make a grand entrance,
Welcome, dear colleague, to Co1t's brilliance!

I'm thrilled to join this team, a fresh face, so bright,
Excited to contribute and learn with all my might.

Let's connect and collaborate, a partnership so rare,
Together, we'll conquer challenges, and success we'll share.

Feel free to reach out, any time, any day,
Let's chat and get to know each other in our own way.

A new adventure awaits, so let's make it grand,
Looking forward to being part of this amazing band!


## Step 3: Streaming the response

The Chat endpoint also provides streaming support. In a streamed response, the endpoint would return a response object for each token as it is being generated. This means you can display the text incrementally without having to wait for the full completion.

To activate it, use `co.chat_stream()` instead of `co.chat()`.

In streaming mode, the endpoint will generate a series of objects. To get the actual text contents, we take objects whose `event_type` is `text-generation`.

In [29]:
# Add the user message
message = "I'm joining a new startup called Co1t today. Could you help me write an introduction message for my teammates."

# Generate the response by streaming it
response = co.chat_stream(model="command-r-plus-08-2024",
                          messages=[{'role':'user', 'content': message}])

for event in response:
    if event.type == "content-delta":
        print(event.delta.message.content.text, end="")

Here's a draft of an introduction message for your new colleagues at Co1t:

"Hello everyone!

My name is [Your Name], and I am thrilled to introduce myself as the newest member of the Co1t team! Starting my journey with you all today, I am excited to be a part of this innovative startup and contribute to its success.

A little about myself: I have [mention your professional background or relevant experience], and I am passionate about [list some of your interests or skills related to the startup's industry]. I believe my expertise in [specific skill or knowledge area] will be a valuable asset to the company's growth.

I am eager to collaborate with each of you and learn from your unique perspectives. As a firm believer in the power of teamwork, I am confident that together, we can achieve great milestones and create something extraordinary.

Looking forward to getting to know you all, brainstorming ideas, and tackling the challenges ahead as a united front. Feel free to reach out if yo

## Step 4: Build the conversation memory

At the core of a conversation is a multi-turn dialog between the user and the chatbot. This requires the chatbot to have the state (or “memory”) of all the previous turns to maintain the state of the conversation.

Let's start with the first turn in the conversation.

Here, we are also adding a custom system message for generating concise response, just to keep the outputs brief for this tutorial.

In [31]:
# Add the user message
message = "I'm joining a new startup called Co1t today. Could you help me write an introduction message for my teammates."

# Create a custom system message
system_message="""## Task and Context
Generate concise responses, with maximum one-sentence."""

# Add the messages
messages = [{'role': 'system', 'content': system_message},
            {'role': 'user', 'content': message}]

# Generate the response
response = co.chat(model="command-r-plus-08-2024",
                   messages=messages)

print(response.message.content[0].text)

"Hi everyone, I'm thrilled to join Co1t as a new team member, and I look forward to collaborating and contributing to our shared success!"


Now, we want the model to refine the earlier response. This requires the next generation to have access to the state, or memory, of the conversation.

To do this, we append the `message` object from the previous response to the `messages` list.

Then we append the next user message to the `messages` list.

Looking at the response, we see that the model is able to get the context from the chat history. The model is able to capture that "it" in the user message refers to the introduction message it had generated earlier.

In [32]:
# Append the previous response
messages.append({'role': 'assistant', 'content': response.message.content})

# Add the user message
message = "Make it more upbeat and conversational."

# Append the user message
messages.append({'role': 'user', 'content': message})

# Generate the response with the current chat history as the context
response = co.chat(model="command-r-plus-08-2024",
                   messages=messages)

print(response.message.content[0].text)

"Hey there, super excited to be a part of the Co1t family! Can't wait to meet you all and dive into some awesome projects together!"


You can continue doing this for any number of turns by repeating the same steps of appending the chatbot and user messages.

In [33]:
# Append the previous response
messages.append({'role': 'assistant', 'content': response.message.content})

# Add the user message
message = "Thanks. Could you create another one for my DM to my manager."

# Append the user message
messages.append({'role': 'user', 'content': message})

# Generate the response with the current chat history as the context
response = co.chat(model="command-r-plus-08-2024",
                   messages=messages)

print(response.message.content[0].text)

"Hey [Manager's Name], just wanted to express my excitement about starting at Co1t today, and I'm grateful for the opportunity to work with you and the team!"


To look at the current chat history, you can print the `messages` list.

In [34]:
# Append the previous response
messages.append({'role': 'assistant', 'content': response.message.content})
# View the chat history
for message in messages:
    print(message,"\n")

{'role': 'system', 'content': '## Task and Context\nGenerate concise responses, with maximum one-sentence.'} 

{'role': 'user', 'content': "I'm joining a new startup called Co1t today. Could you help me write an introduction message for my teammates."} 

{'role': 'assistant', 'content': [TextAssistantMessageResponseContentItem(type='text', text='"Hi everyone, I\'m thrilled to join Co1t as a new team member, and I look forward to collaborating and contributing to our shared success!"')]} 

{'role': 'user', 'content': 'Make it more upbeat and conversational.'} 

{'role': 'assistant', 'content': [TextAssistantMessageResponseContentItem(type='text', text='"Hey there, super excited to be a part of the Co1t family! Can\'t wait to meet you all and dive into some awesome projects together!"')]} 

{'role': 'user', 'content': 'Thanks. Could you create another one for my DM to my manager.'} 

{'role': 'assistant', 'content': [TextAssistantMessageResponseContentItem(type='text', text='"Hey [Manage